In [1]:
import os 
os.chdir("../")
%pwd

'c:\\Users\\josep\\OneDrive\\Desktop\\Chest-Cancer-Classification-using-MLflow-and-DVC'

In [3]:
%pip install dagshub

  Obtaining dependency information for dagshub from https://files.pythonhosted.org/packages/ce/7e/b2b4cfc8ca4dc38377ac02237b7ad547b34fe07ef057b3a52c3f01d934c7/dagshub-0.3.34-py3-none-any.whl.metadata
  Using cached dagshub-0.3.34-py3-none-any.whl.metadata (11 kB)
  Using cached fusepy-3.0.1-py3-none-any.whl
  Obtaining dependency information for httpx~=0.23.0 from https://files.pythonhosted.org/packages/ac/a2/0260c0f5d73bdf06e8d3fc1013a82b9f0633dc21750c9e3f3cb1dba7bb8c/httpx-0.23.3-py3-none-any.whl.metadata
  Using cached httpx-0.23.3-py3-none-any.whl.metadata (7.1 kB)
  Obtaining dependency information for rich~=13.1.0 from https://files.pythonhosted.org/packages/55/19/8b1ed0f3ea49306b8115afe84e8e5cd92925d732260efc75e4e3e3089bf0/rich-13.1.0-py3-none-any.whl.metadata
  Using cached rich-13.1.0-py3-none-any.whl.metadata (18 kB)
  Obtaining dependency information for dacite~=1.6.0 from https://files.pythonhosted.org/packages/06/9d/11a073172d889e9e0d0ad270a1b468876c82d759af7864a8095dfc73f

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.2.4 requires httpx>=0.25.0, but you have httpx 0.23.3 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
Mlflow_tracking_uri = "https://dagshub.com/josephkj47/Chest-Cancer-Classification-using-MLflow-and-DVC.mlflow"
import dagshub
dagshub.init(repo_owner='josephkj47', repo_name='Chest-Cancer-Classification-using-MLflow-and-DVC', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as josephkj47

Initialized MLflow to track repo "josephkj47/Chest-Cancer-Classification-using-MLflow-and-DVC"

Repository josephkj47/Chest-Cancer-Classification-using-MLflow-and-DVC initialized!

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/CT-Scan images",
            mlflow_uri="https://dagshub.com/josephkj47/Chest-Cancer-Classification-using-MLflow-and-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [9]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [11]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-08-23 03:03:37,434: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-23 03:03:37,434: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-23 03:03:37,434: INFO: common: created directory at: artifacts]
Found 109 images belonging to 1 classes.
7/7 [==============================] - 7s 969ms/step - loss: 56.9461 - accuracy: 0.0000e+00
[2024-08-23 03:03:44,624: INFO: common: json file saved at: scores.json]


2024/08/23 03:03:45 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-08-23 03:03:45,756: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\josep\AppData\Local\Temp\tmp7kb7aka0\model\data\model\assets
[2024-08-23 03:03:46,070: INFO: builder_impl: Assets written to: C:\Users\josep\AppData\Local\Temp\tmp7kb7aka0\model\data\model\assets]


Successfully registered model 'VGG16Model'.
2024/08/23 03:04:12 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
